In [399]:
import pandas as pd
from sklearn.preprocessing import  MinMaxScaler, OneHotEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from scipy.stats import randint
Data_train = pd.read_csv('train.csv')
Data_test = pd.read_csv('test.csv')


In [400]:
print(Data_train.shape)
print(Data_test.shape)

(891, 12)
(418, 11)


In [401]:
Data_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [402]:
Data_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [403]:
Data_train['Title'] = Data_train['Name'].str.extract(r',\s*([^\.]+)\.', expand=False)
Data_train['Cabin_First_Letter'] = Data_train['Cabin'].fillna('Unknown').str[0]

In [404]:
df_PassengerId = Data_test['PassengerId']
Data_train = Data_train.drop(columns=['Name', 'Cabin', 'Ticket', 'PassengerId'],axis=1)
Data_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Cabin_First_Letter
0,0,3,male,22.0,1,0,7.2500,S,Mr,U
1,1,1,female,38.0,1,0,71.2833,C,Mrs,C
2,1,3,female,26.0,0,0,7.9250,S,Miss,U
3,1,1,female,35.0,1,0,53.1000,S,Mrs,C
4,0,3,male,35.0,0,0,8.0500,S,Mr,U


In [405]:
Data_test['Title'] = Data_test['Name'].str.extract(r',\s*([^\.]+)\.', expand=False)
Data_test['Cabin_First_Letter'] = Data_test['Cabin'].fillna('Unknown').str[0]

In [406]:
Data_test = Data_test.drop(columns=['Name', 'Cabin', 'Ticket', 'PassengerId'],axis=1)
Data_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Cabin_First_Letter
0,3,male,34.5,0,0,7.8292,Q,Mr,U
1,3,female,47.0,1,0,7.0000,S,Mrs,U
2,2,male,62.0,0,0,9.6875,Q,Mr,U
3,3,male,27.0,0,0,8.6625,S,Mr,U
4,3,female,22.0,1,1,12.2875,S,Mrs,U


In [407]:
Columns = ['Sex','Title','Cabin_First_Letter', 'Embarked']
Encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_train_encode = pd.DataFrame(Encoder.fit_transform(Data_train[Columns]), columns= Encoder.get_feature_names_out(Columns))
X_test_encode = pd.DataFrame(Encoder.transform(Data_test[Columns]), columns= Encoder.get_feature_names_out(Columns))

In [408]:
X_train_encode

,Sex_female,Sex_male,Title_Capt,Title_Col,Title_Don,Title_Dr,Title_Jonkheer,Title_Lady,Title_Major,Title_Master,...,Cabin_First_Letter_D,Cabin_First_Letter_E,Cabin_First_Letter_F,Cabin_First_Letter_G,Cabin_First_Letter_T,Cabin_First_Letter_U,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
887,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
888,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
889,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [409]:
Data_train = Data_train.drop(columns=Columns)
Data_test = Data_test.drop(columns=Columns)

Data_train = pd.concat([Data_train, X_train_encode], axis=1)
Data_test = pd.concat([Data_test, X_test_encode], axis=1)

In [410]:

continuous_columns = [ 'Age', 'SibSp', 'Parch', 'Fare']

scaler = MinMaxScaler()
Data_train[continuous_columns] = scaler.fit_transform(Data_train[continuous_columns])
Data_test[continuous_columns] = scaler.transform(Data_test[continuous_columns])

In [411]:
Data_train

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Title_Capt,Title_Col,...,Cabin_First_Letter_D,Cabin_First_Letter_E,Cabin_First_Letter_F,Cabin_First_Letter_G,Cabin_First_Letter_T,Cabin_First_Letter_U,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,0,3,0.271204,0.125,0.000000,0.014151,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,1,1,0.472251,0.125,0.000000,0.139136,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1,3,0.321466,0.000,0.000000,0.015469,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,1,1,0.434555,0.125,0.000000,0.103644,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0,3,0.434555,0.000,0.000000,0.015713,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,0.334031,0.000,0.000000,0.025374,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
887,1,1,0.233508,0.000,0.000000,0.058556,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
888,0,3,NaN,0.125,0.333333,0.045771,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
889,1,1,0.321466,0.000,0.000000,0.058556,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [412]:
print(Data_train.isna().sum())

Survived                  0
Pclass                    0
Age                     177
SibSp                     0
Parch                     0
Fare                      0
Sex_female                0
Sex_male                  0
Title_Capt                0
Title_Col                 0
Title_Don                 0
Title_Dr                  0
Title_Jonkheer            0
Title_Lady                0
Title_Major               0
Title_Master              0
Title_Miss                0
Title_Mlle                0
Title_Mme                 0
Title_Mr                  0
Title_Mrs                 0
Title_Ms                  0
Title_Rev                 0
Title_Sir                 0
Title_the Countess        0
Cabin_First_Letter_A      0
Cabin_First_Letter_B      0
Cabin_First_Letter_C      0
Cabin_First_Letter_D      0
Cabin_First_Letter_E      0
Cabin_First_Letter_F      0
Cabin_First_Letter_G      0
Cabin_First_Letter_T      0
Cabin_First_Letter_U      0
Embarked_C                0
Embarked_Q          

In [413]:
print(Data_test.isna().sum())

Pclass                   0
Age                     86
SibSp                    0
Parch                    0
Fare                     1
Sex_female               0
Sex_male                 0
Title_Capt               0
Title_Col                0
Title_Don                0
Title_Dr                 0
Title_Jonkheer           0
Title_Lady               0
Title_Major              0
Title_Master             0
Title_Miss               0
Title_Mlle               0
Title_Mme                0
Title_Mr                 0
Title_Mrs                0
Title_Ms                 0
Title_Rev                0
Title_Sir                0
Title_the Countess       0
Cabin_First_Letter_A     0
Cabin_First_Letter_B     0
Cabin_First_Letter_C     0
Cabin_First_Letter_D     0
Cabin_First_Letter_E     0
Cabin_First_Letter_F     0
Cabin_First_Letter_G     0
Cabin_First_Letter_T     0
Cabin_First_Letter_U     0
Embarked_C               0
Embarked_Q               0
Embarked_S               0
Embarked_nan             0
d

In [414]:
X_train = Data_train.drop(columns=['Survived'])
Y_train = Data_train['Survived']

X_test = Data_test

In [415]:
impute = IterativeImputer(initial_strategy='median', random_state=60)
X_train = pd.DataFrame(impute.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(impute.transform(X_test), columns=X_test.columns)

In [416]:
print(X_train.head())
print(X_train.isna().sum())
print(X_test.isna().sum())

   Pclass       Age  SibSp  Parch      Fare  Sex_female  Sex_male  Title_Capt  \
0     3.0  0.271204  0.125    0.0  0.014151         0.0       1.0         0.0   
1     1.0  0.472251  0.125    0.0  0.139136         1.0       0.0         0.0   
2     3.0  0.321466  0.000    0.0  0.015469         1.0       0.0         0.0   
3     1.0  0.434555  0.125    0.0  0.103644         1.0       0.0         0.0   
4     3.0  0.434555  0.000    0.0  0.015713         0.0       1.0         0.0   

   Title_Col  Title_Don  ...  Cabin_First_Letter_D  Cabin_First_Letter_E  \
0        0.0        0.0  ...                   0.0                   0.0   
1        0.0        0.0  ...                   0.0                   0.0   
2        0.0        0.0  ...                   0.0                   0.0   
3        0.0        0.0  ...                   0.0                   0.0   
4        0.0        0.0  ...                   0.0                   0.0   

   Cabin_First_Letter_F  Cabin_First_Letter_G  Cabin_Fir

In [417]:
modelo_logistic = cross_val_score(LogisticRegression(), X_train, Y_train, cv =2)
modelo_logistic

array([0.79372197, 0.8       ])

In [418]:
modelo_SVC = cross_val_score(SVC(), X_train, Y_train, cv =2)
modelo_SVC

array([0.77802691, 0.79325843])

In [419]:
modelo_KNeighborsClassifier = cross_val_score(KNeighborsClassifier(), X_train, Y_train, cv =2)
modelo_KNeighborsClassifier

array([0.80044843, 0.78876404])

In [420]:
modelo_GaussianNB = cross_val_score(GaussianNB(), X_train, Y_train, cv =2)
modelo_GaussianNB

array([0.69058296, 0.40224719])

In [421]:
modelo_GradientBoostingClassifier = cross_val_score(GradientBoostingClassifier(), X_train, Y_train, cv =2)
modelo_GradientBoostingClassifier

array([0.77578475, 0.81348315])

In [422]:
modelo_RandomForestClassifier = cross_val_score(RandomForestClassifier(), X_train, Y_train, cv =2)
modelo_RandomForestClassifier

array([0.79820628, 0.78202247])

In [423]:
param_dist = {
    'n_estimators': randint(50, 200),
    'learning_rate': [0.001,0.01,0.02,0.01,0.2],
    'max_depth': randint(1, 10),
    'subsample': [0.3, 0.5, 0.7, 1.0],
    'min_samples_split': randint(2, 30),
    'min_samples_leaf': randint(1, 10)
}

In [424]:
finding_model = GradientBoostingClassifier(random_state=60)

In [425]:
random_search = RandomizedSearchCV(finding_model, param_distributions=param_dist, n_iter=100, cv=3, 
                                   scoring='accuracy', n_jobs=-1, verbose=3, random_state=60 )

random_search.fit(X_train, Y_train)

best_params = random_search.best_params_
print("Best Parameters:", best_params)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best Parameters: {'learning_rate': 0.01, 'max_depth': 5, 'min_samples_leaf': 7, 'min_samples_split': 4, 'n_estimators': 191, 'subsample': 0.5}


In [426]:
print("Best Parameters:", random_search.best_score_)

Best Parameters: 0.8406285072951739


In [427]:
Final = GradientBoostingClassifier(**best_params)
Final.fit(X_train,Y_train)
Pred = Final.predict(X_test)

In [431]:
Pred_df = pd.DataFrame({'PassengerId': df_PassengerId, 'Survived': Pred})
Pred_df['Survived'] = Pred_df['Survived'].map({0: 'Not Survived', 1: 'Survived'})
print(Pred_df)

     PassengerId      Survived
0            892  Not Survived
1            893  Not Survived
2            894  Not Survived
3            895  Not Survived
4            896      Survived
..           ...           ...
413         1305  Not Survived
414         1306      Survived
415         1307  Not Survived
416         1308  Not Survived
417         1309      Survived

[418 rows x 2 columns]
